In [4]:
from Funcs.Utility import *

# Cross-validation

## Implementation

### CV Pipeline

In [1]:
import os
import pandas as pd
import numpy as np
import traceback as tb
from contextlib import contextmanager
from typing import Tuple, Dict, Union, Generator, List
from dataclasses import dataclass
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.base import BaseEstimator, clone
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut, StratifiedShuffleSplit, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
import time
import ray


@dataclass
class FoldResult:
    name: str
    estimator: BaseEstimator
    X_train: pd.DataFrame
    y_train: np.ndarray
    X_test: pd.DataFrame
    y_test: np.ndarray
    categories: Dict[str, Dict[int, str]] = None


def _split(
        alg: str,
        X: Union[pd.DataFrame, np.ndarray] = None,
        y: np.ndarray = None,
        groups: np.ndarray = None,
        random_state: int = None,
        n_splits: int = None,
        n_repeats: int = None,
        test_ratio: float = None
) -> Generator[Tuple[np.ndarray, np.ndarray], None, None]:
    if alg == 'holdout':
        splitter = StratifiedShuffleSplit(
            n_splits=n_splits,
            test_size=test_ratio,
            random_state=random_state
        )
    elif alg == 'kfold':
        if n_repeats and n_repeats > 1:
            splitter = RepeatedStratifiedKFold(
                n_splits=n_splits,
                n_repeats=n_repeats,
                random_state=random_state,
            )
        else:
            splitter = StratifiedKFold(
                n_splits=n_splits,
                random_state=random_state,
                shuffle=False if random_state is None else True,
            )
    elif alg == 'logo':
        splitter = LeaveOneGroupOut()
    else:
        raise ValueError('"alg" should be one of "holdout", "kfold", "logo", or "groupk".')

    split = splitter.split(X, y, groups)

    for I_train, I_test in split:
        yield I_train, I_test


def _train(
    dir_result: str,
    name: str,
    X_train: pd.DataFrame,
    y_train: np.ndarray,
    X_test: pd.DataFrame,
    y_test: np.ndarray,
    C_cat: np.ndarray,
    C_num: np.ndarray,
    estimator: BaseEstimator,
    normalize: bool = False,
    select: Union[List[SelectFromModel], SelectFromModel] = None,
    oversample: bool = False,
    random_state: int = None,
    categories: Union[List, Dict[str, Dict[int, str]]] = None
):
    @contextmanager
    def _log(task_type: str):
        log(f'In progress: {task_type}.')
        _t = time.time()
        _err = None
        _result = dict()
        
        try:
            yield _result
        except:
            _err = tb.format_exc()
        finally:
            _e = time.time() - _t
            if _err:
                _msg = f'Failure: {task_type} ({_e:.2f}s). Keep running without this task. Caused by: \n{_err}' 
            else:
                _msg = f'Success: {task_type} ({_e:.2f}s).' 
                if _result:
                    _r = '\n'.join([f'- {k}: {v}' for k, v in _result.items()])
                    _msg = f'{_msg}\n{_r}'
            log(_msg)
    
    if normalize:
        with _log(f'[{name}] Normalizing numeric features'):
            X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
            X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values
            
            scaler = StandardScaler().fit(X_train_N)
            X_train_N = scaler.transform(X_train_N)
            X_test_N = scaler.transform(X_test_N)
         
            X_train = pd.DataFrame(
                np.concatenate((X_train_C, X_train_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
            X_test = pd.DataFrame(
                np.concatenate((X_test_C, X_test_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
           
    if select:
        if isinstance(select, SelectFromModel):
            select = [select]
            
        for i, s in enumerate(select):
            with _log(f'[{name}] {i+1}-th Feature selection') as r:
                C = np.asarray(X_train.columns)
                r['# Orig. Feat.'] = f'{len(C)} (# Cat. = {len(C_cat)}; # Num. = {len(C_num)})'
                M = s.fit(X=X_train.values, y=y_train).get_support()
                C_sel = C[M]
                C_cat = C_cat[np.isin(C_cat, C_sel)]
                C_num = C_num[np.isin(C_num, C_sel)]
                
                X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
                X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values


                X_train = pd.DataFrame(
                    np.concatenate((X_train_C, X_train_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )
                X_test = pd.DataFrame(
                    np.concatenate((X_test_C, X_test_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )
                r['# Sel. Feat.'] = f'{len(C_sel)} (# Cat. = {len(C_cat)}; # Num. = {len(C_num)})'

    if oversample:
        with _log(f'[{name}] Oversampling') as r:
            if len(C_cat):
                M = np.isin(X_train.columns, C_cat)
                sampler = SMOTENC(categorical_features=M, random_state=random_state)
            else:
                sampler = SMOTE(random_state=random_state)
            X_train, y_train = sampler.fit_resample(X_train, y_train)

    with _log(f'[{name}] Training'):
        estimator = estimator.fit(X_train, y_train)
        result = FoldResult(
            name=name,
            estimator=estimator,
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            y_test=y_test,
            categories=categories
        )
        dump(result, os.path.join(dir_result, f'{name}.pkl'))
    

def cross_val(
    X: pd.DataFrame,
    y: np.ndarray,
    groups: np.ndarray,
    path: str,
    name: str,
    estimator: BaseEstimator,
    categories: List[str] = None,
    normalize: bool = False,
    split: str = None,
    split_params: Dict[str, any] = None,
    select: Union[List[SelectFromModel], SelectFromModel] = None,
    oversample: bool = False,
    random_state: int = None
):
    if not os.path.exists(path):
        raise ValueError('"path" does not exist.')
    
    if not split:
        raise ValueError('"split" should be specified.')
    
    if not ray.is_initialized():
        raise EnvironmentError('"ray" should be initialized.')
    
    jobs = []
    func = ray.remote(_train).remote

    categories = list() if categories is None else categories
    C_cat = np.asarray(sorted(categories))
    C_num = np.asarray(sorted(X.columns[~X.columns.isin(C_cat)]))

    split_params = split_params or dict()
    splitter = _split(alg=split, X=X, y=y, groups=groups, random_state=random_state, **split_params)

    for idx_fold, (I_train, I_test) in enumerate(splitter):
        if split == 'logo':
            FOLD_NAME = str(np.unique(groups[I_test]).item(0))
        else:
            FOLD_NAME = str(idx_fold + 1)

        X_train, y_train = X.iloc[I_train, :], y[I_train]
        X_test, y_test = X.iloc[I_test, :], y[I_test]

        job = func(
            dir_result=path,
            name=f'{name}#{FOLD_NAME}',
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            y_test=y_test,
            C_cat=C_cat,
            C_num=C_num,
            categories=categories,
            estimator=clone(estimator),
            normalize=normalize,
            select=select,
            oversample=oversample,
            random_state=random_state
        )
        jobs.append(job)
    ray.get(jobs)

### Minor Modification on XGBClassifer
This modification allows XGBClassifiers to automatically generate evaluation sets during pipeline (without passing any argument in "fit" function)

In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator
from sklearn.model_selection import StratifiedShuffleSplit
from typing import Union


class EvXGBClassifier(BaseEstimator):
    def __init__(
        self,
        eval_size=None,
        eval_metric='logloss',
        early_stopping_rounds=10,
        random_state=None,
        **kwargs
        ):
        self.random_state = random_state
        self.eval_size = eval_size
        self.eval_metric = eval_metric
        self.early_stopping_rounds = early_stopping_rounds
        self.model = XGBClassifier(
            random_state=self.random_state,
            eval_metric=self.eval_metric,
            early_stopping_rounds=self.early_stopping_rounds,
            **kwargs
        )

    @property
    def classes_(self):
        return self.model.classes_

    @property
    def feature_importances_(self):
        return self.model.feature_importances_
    
    @property
    def feature_names_in_(self):
        return self.model.feature_names_in_

    def fit(self, X: Union[pd.DataFrame, np.ndarray], y: np.ndarray):
        if self.eval_size:
            splitter = StratifiedShuffleSplit(random_state=self.random_state, test_size=self.eval_size)
            I_train, I_eval = next(splitter.split(X, y))
            if isinstance(X, pd.DataFrame):
                X_train, y_train = X.iloc[I_train, :], y[I_train]
                X_eval, y_eval = X.iloc[I_eval, :], y[I_eval]
            else:
                X_train, y_train = X[I_train, :], y[I_train]
                X_eval, y_eval = X[I_eval, :], y[I_eval]
                
            self.model = self.model.fit(
                X=X_train, y=y_train, 
                eval_set=[(X_eval, y_eval)],
                verbose=False
            )
        else:
            self.model = self.model.fit(X=X, y=y, verbose=False)
        return self

    def predict(self, X: pd.DataFrame):
        return self.model.predict(X)

    def predict_proba(self, X: pd.DataFrame):
        return self.model.predict_proba(X)

## Execution

In [7]:
import os
from itertools import product
from sklearn.base import clone
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from eli5.sklearn.permutation_importance import PermutationImportance



RANDOM_STATE = 42

ESTIMATOR_DUMMY = DummyClassifier(strategy='prior')
ESTIMATOR_RF = RandomForestClassifier(random_state=RANDOM_STATE)
ESTIMATOR_XGB = EvXGBClassifier(
    random_state=RANDOM_STATE, 
    eval_metric='logloss', 
    eval_size=0.2,
    early_stopping_rounds=10, 
    objective='binary:logistic', 
    verbosity=0,
    learning_rate=0.01
)

SELECT_SVC = SelectFromModel(
    estimator=LinearSVC(
        penalty='l1',
        loss='squared_hinge',
        dual=False,
        tol=1e-3,
        C=1e-2,
        max_iter=5000,
        random_state=RANDOM_STATE
    ),
    threshold=1e-5
)

# CLS = ['valence', 'arousal', 'stress', 'disturbance']
CLS = ['stress']
SETTINGS = [
    dict(
        estimator=clone(ESTIMATOR_DUMMY),
        oversample=False,
        select=None,
        name='dummy'
    ),
    dict(
        estimator=clone(ESTIMATOR_RF),
        oversample=False,
        select=[clone(SELECT_SVC)],
        name='rf_ns'
    ),
#     dict(
#         estimator=clone(ESTIMATOR_RF),
#         oversample=True,
#         select=[clone(SELECT_SVC)],
#         name='rf_os'
#     ),
    dict(
        estimator=clone(ESTIMATOR_XGB),
        oversample=False,
        select=[clone(SELECT_SVC)],
        name='xgb_ns'
    ),
#     dict(
#         estimator=clone(ESTIMATOR_XGB),
#         oversample=True,
#         select=[clone(SELECT_SVC)],
#         name='xgb_os'
#     )
]

with on_ray(num_cpus=8):
    for l, s in product(
        CLS, SETTINGS
    ):
        p = os.path.join(PATH_INTERMEDIATE, 'feat',f'{l}.pkl')
        par_dir = os.path.join(PATH_INTERMEDIATE, 'eval', l)
        os.makedirs(par_dir, exist_ok=True)
        
        X, y, groups, t, datetimes = load(p)
        cats = X.columns[X.dtypes == bool]
        cross_val(
            X=X, y=y, groups=groups,
            path=par_dir,
            categories=cats,
            normalize=True,
            split='logo',
            random_state=RANDOM_STATE,
            **s
        )

2023-05-16 14:07:28,287	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(_train pid=252144) [23-05-16 14:07:29] In progress: [dummy#P01] Normalizing numeric features.
(_train pid=252143) [23-05-16 14:07:30] In progress: [dummy#P06] Normalizing numeric features.
(_train pid=252148) [23-05-16 14:07:30] In progress: [dummy#P02] Normalizing numeric features.
(_train pid=252150) [23-05-16 14:07:30] In progress: [dummy#P10] Normalizing numeric features.
(_train pid=252146) [23-05-16 14:07:30] In progress: [dummy#P03] Normalizing numeric features.
(_train pid=252145) [23-05-16 14:07:30] In progress: [dummy#P05] Normalizing numeric features.
(_train pid=252144) [23-05-16 14:07:30] Success: [dummy#P01] Normalizing numeric features (0.38s).
(_train pid=252144) [23-05-16 14:07:30] In progress: [dummy#P01] Training.
(_train pid=252149) [23-05-16 14:07:30] In progress: [dummy#P08] Normalizing numeric features.
(_train pid=252143) [23-05-16 14:07:30] Success: [dummy#P06] Normalizing numeric features (0.47s).
(_train pid=252143) [23-05-16 14:07:30] In progress: [dummy#P0

(_train pid=252150) [23-05-16 14:07:35] Success: [dummy#P35] Normalizing numeric features (0.25s).
(_train pid=252150) [23-05-16 14:07:35] In progress: [dummy#P35] Training.
(_train pid=252149) [23-05-16 14:07:35] In progress: [dummy#P47] Normalizing numeric features.
(_train pid=252150) [23-05-16 14:07:35] Success: [dummy#P35] Training (0.06s).
(_train pid=252150) [23-05-16 14:07:36] In progress: [dummy#P48] Normalizing numeric features.
(_train pid=252146) [23-05-16 14:07:35] Success: [dummy#P39] Normalizing numeric features (0.28s).
(_train pid=252146) [23-05-16 14:07:35] In progress: [dummy#P39] Training.
(_train pid=252146) [23-05-16 14:07:36] Success: [dummy#P39] Training (0.05s).
(_train pid=252146) [23-05-16 14:07:36] In progress: [dummy#P49] Normalizing numeric features.
(_train pid=252143) [23-05-16 14:07:36] Success: [dummy#P13] Training (2.65s).
(_train pid=252143) [23-05-16 14:07:36] In progress: [dummy#P50] Normalizing numeric features.
(_train pid=252144) [23-05-16 14:07

(_train pid=252150) [23-05-16 14:07:37] Success: [dummy#P76] Normalizing numeric features (0.59s).
(_train pid=252150) [23-05-16 14:07:37] In progress: [dummy#P76] Training.
(_train pid=252143) [23-05-16 14:07:37] Success: [dummy#P77] Normalizing numeric features (0.42s).
(_train pid=252143) [23-05-16 14:07:37] In progress: [dummy#P77] Training.
(_train pid=252143) [23-05-16 14:07:37] Success: [dummy#P77] Training (0.05s).
(_train pid=252150) [23-05-16 14:07:37] Success: [dummy#P76] Training (0.05s).
(_train pid=252146) [23-05-16 14:07:37] Success: [dummy#P78] Normalizing numeric features (0.44s).
(_train pid=252146) [23-05-16 14:07:37] In progress: [dummy#P78] Training.
(_train pid=252146) [23-05-16 14:07:37] Success: [dummy#P78] Training (0.04s).
(_train pid=252149) [23-05-16 14:07:37] Success: [dummy#P80] Normalizing numeric features (0.37s).
(_train pid=252149) [23-05-16 14:07:37] In progress: [dummy#P80] Training.
(_train pid=252149) [23-05-16 14:07:37] Success: [dummy#P80] Traini

(_train pid=252143) [23-05-16 14:07:43] Success: [rf_ns#P15] Normalizing numeric features (0.32s).
(_train pid=252143) [23-05-16 14:07:43] In progress: [rf_ns#P15] 1-th Feature selection.
(_train pid=252144) [23-05-16 14:07:43] Success: [rf_ns#P10] Training (1.39s).
(_train pid=252144) [23-05-16 14:07:43] In progress: [rf_ns#P28] Normalizing numeric features.
(_train pid=252148) [23-05-16 14:07:43] Success: [rf_ns#P09] Training (1.35s).
(_train pid=252148) [23-05-16 14:07:43] In progress: [rf_ns#P26] Normalizing numeric features.
(_train pid=252146) [23-05-16 14:07:43] Success: [rf_ns#P03] Training (1.36s).
(_train pid=252146) [23-05-16 14:07:43] In progress: [rf_ns#P23] Normalizing numeric features.
(_train pid=252147) [23-05-16 14:07:43] Success: [rf_ns#P19] Normalizing numeric features (0.40s).
(_train pid=252147) [23-05-16 14:07:43] In progress: [rf_ns#P19] 1-th Feature selection.
(_train pid=252149) [23-05-16 14:07:43] Success: [rf_ns#P21] Normalizing numeric features (0.43s).
(_t

...

(_train pid=252143) [23-05-16 14:07:57] Success: [rf_ns#P60] Training (1.56s).
(_train pid=252143) [23-05-16 14:07:57] In progress: [rf_ns#P80] Normalizing numeric features.
(_train pid=252145) [23-05-16 14:07:58] Success: [rf_ns#P72] 1-th Feature selection (1.63s).
(_train pid=252145) - # Orig. Feat.: 6220 (# Cat. = 60; # Num. = 6160)
(_train pid=252145) - # Sel. Feat.: 225 (# Cat. = 3; # Num. = 222)
(_train pid=252145) [23-05-16 14:07:58] In progress: [rf_ns#P72] Training.
(_train pid=252145) 
(_train pid=252143) [23-05-16 14:07:58] Success: [rf_ns#P80] Normalizing numeric features (0.30s).
(_train pid=252143) [23-05-16 14:07:58] In progress: [rf_ns#P80] 1-th Feature selection.
(_train pid=252146) [23-05-16 14:07:57] Success: [rf_ns#P79] Normalizing numeric features (0.37s).
(_train pid=252146) [23-05-16 14:07:58] In progress: [rf_ns#P79] 1-th Feature selection.
(_train pid=252149) [23-05-16 14:07:58] Success: [rf_ns#P70] Training (1.57s).
(_train pid=252147) [23-05-16 14:07:58] Succ

(_train pid=252149) [23-05-16 14:08:05] Success: [xgb_ns#P10] Training (1.25s).
(_train pid=252149) [23-05-16 14:08:05] In progress: [xgb_ns#P12] Normalizing numeric features.
(_train pid=252144) [23-05-16 14:08:05] Success: [xgb_ns#P06] Training (1.28s).
(_train pid=252144) [23-05-16 14:08:05] In progress: [xgb_ns#P13] Normalizing numeric features.
(_train pid=252143) [23-05-16 14:08:05] Success: [xgb_ns#P01] Training (1.43s).
(_train pid=252146) [23-05-16 14:08:05] Success: [xgb_ns#P05] Training (1.22s).
(_train pid=252146) [23-05-16 14:08:05] In progress: [xgb_ns#P15] Normalizing numeric features.
(_train pid=252143) [23-05-16 14:08:05] In progress: [xgb_ns#P19] Normalizing numeric features.
(_train pid=252144) [23-05-16 14:08:05] Success: [xgb_ns#P13] Normalizing numeric features (0.29s).
(_train pid=252144) [23-05-16 14:08:05] In progress: [xgb_ns#P13] 1-th Feature selection.
(_train pid=252150) [23-05-16 14:08:05] Success: [xgb_ns#P03] Training (1.79s).
(_train pid=252150) [23-05

(_train pid=252146) [23-05-16 14:08:11] Success: [xgb_ns#P35] 1-th Feature selection (2.01s).
(_train pid=252146) - # Orig. Feat.: 6220 (# Cat. = 60; # Num. = 6160)
(_train pid=252146) - # Sel. Feat.: 225 (# Cat. = 2; # Num. = 223)
(_train pid=252146) [23-05-16 14:08:11] In progress: [xgb_ns#P35] Training.
(_train pid=252143) [23-05-16 14:08:11] Success: [xgb_ns#P33] 1-th Feature selection (2.13s).
(_train pid=252143) - # Orig. Feat.: 6220 (# Cat. = 60; # Num. = 6160)
(_train pid=252143) - # Sel. Feat.: 223 (# Cat. = 3; # Num. = 220)
(_train pid=252143) [23-05-16 14:08:11] In progress: [xgb_ns#P33] Training.
(_train pid=252150) [23-05-16 14:08:11] Success: [xgb_ns#P42] 1-th Feature selection (2.04s).
(_train pid=252150) - # Orig. Feat.: 6220 (# Cat. = 60; # Num. = 6160)
(_train pid=252150) - # Sel. Feat.: 229 (# Cat. = 2; # Num. = 227)
(_train pid=252150) [23-05-16 14:08:11] In progress: [xgb_ns#P42] Training.
(_train pid=252144) [23-05-16 14:08:12] Success: [xgb_ns#P31] 1-th Feature s

(_train pid=252143) [23-05-16 14:08:15] Success: [xgb_ns#P49] Training (0.53s).
(_train pid=252143) [23-05-16 14:08:15] In progress: [xgb_ns#P57] Normalizing numeric features.
(_train pid=252146) [23-05-16 14:08:15] Success: [xgb_ns#P47] Training (0.57s).
(_train pid=252146) [23-05-16 14:08:15] In progress: [xgb_ns#P55] Normalizing numeric features.
(_train pid=252145) [23-05-16 14:08:15] Success: [xgb_ns#P50] 1-th Feature selection (2.18s).
(_train pid=252145) - # Orig. Feat.: 6220 (# Cat. = 60; # Num. = 6160)
(_train pid=252145) - # Sel. Feat.: 224 (# Cat. = 3; # Num. = 221)
(_train pid=252145) [23-05-16 14:08:15] In progress: [xgb_ns#P50] Training.
(_train pid=252144) [23-05-16 14:08:15] Success: [xgb_ns#P48] Training (0.60s).
(_train pid=252144) [23-05-16 14:08:15] In progress: [xgb_ns#P60] Normalizing numeric features.
(_train pid=252143) [23-05-16 14:08:15] Success: [xgb_ns#P57] Normalizing numeric features (0.23s).
(_train pid=252143) [23-05-16 14:08:15] In progress: [xgb_ns#P57

(_train pid=252150) [23-05-16 14:08:19] Success: [xgb_ns#P61] Training (1.00s).
(_train pid=252150) [23-05-16 14:08:19] In progress: [xgb_ns#P78] Normalizing numeric features.
(_train pid=252149) [23-05-16 14:08:19] Success: [xgb_ns#P77] Normalizing numeric features (0.32s).
(_train pid=252149) [23-05-16 14:08:19] In progress: [xgb_ns#P77] 1-th Feature selection.
(_train pid=252147) [23-05-16 14:08:19] Success: [xgb_ns#P67] Training (0.60s).
(_train pid=252147) [23-05-16 14:08:19] In progress: [xgb_ns#P79] Normalizing numeric features.
(_train pid=252150) [23-05-16 14:08:19] Success: [xgb_ns#P78] Normalizing numeric features (0.30s).
(_train pid=252150) [23-05-16 14:08:19] In progress: [xgb_ns#P78] 1-th Feature selection.
(_train pid=252148) [23-05-16 14:08:19] Success: [xgb_ns#P70] Training (0.89s).
(_train pid=252145) [23-05-16 14:08:19] Success: [xgb_ns#P69] Training (0.74s).
(_train pid=252145) [23-05-16 14:08:19] In progress: [xgb_ns#P80] Normalizing numeric features.
(_train pid=

In [26]:
X, y, groups, t, datetimes = load(p)

In [38]:
X

,PIF#AGE,PIF#BFI_OPN,PIF#BFI_CON,PIF#BFI_NEU,PIF#BFI_EXT,PIF#BFI_AGR,PIF#PSS10,PIF#PHQ9,PIF#GHQ12,PIF#GEN=F,...,MED_VID#MEDTodayEvening,MED_VID#TSCTodayEvening,MED_VID#AVGTodayNight,MED_VID#STDTodayNight,MED_VID#SKWTodayNight,MED_VID#KURTodayNight,MED_VID#ASCTodayNight,MED_VID#BEPTodayNight,MED_VID#MEDTodayNight,MED_VID#TSCTodayNight
0,27.0,11.0,11.0,3.0,4.0,13.0,13.0,0.0,1.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,27.0,11.0,11.0,3.0,4.0,13.0,13.0,0.0,1.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,27.0,11.0,11.0,3.0,4.0,13.0,13.0,0.0,1.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27.0,11.0,11.0,3.0,4.0,13.0,13.0,0.0,1.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,27.0,11.0,11.0,3.0,4.0,13.0,13.0,0.0,1.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2614,25.0,13.0,7.0,5.0,4.0,12.0,29.0,15.0,15.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2615,25.0,13.0,7.0,5.0,4.0,12.0,29.0,15.0,15.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2616,25.0,13.0,7.0,5.0,4.0,12.0,29.0,15.0,15.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2617,25.0,13.0,7.0,5.0,4.0,12.0,29.0,15.0,15.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
X['ESM#LastLabel'].value_counts()

-1.0    2619
Name: ESM#LastLabel, dtype: int64

In [23]:
feat_current = X.loc[:,[('#VAL' in str(x)) for x in X.keys()] or [('ESM#LastLabel' in str(x)) for x in X.keys()]]

In [25]:
feat_current.keys()

Index(['ACC_AXX#VAL', 'ACC_AXY#VAL', 'ACC_AXZ#VAL', 'ACC_MAG#VAL', 'AML#VAL',
       'CAL#VAL', 'DST_DST#VAL', 'DST_MOT#VAL=IDLE', 'DST_MOT#VAL=WALKING',
       'DST_MOT#VAL=JOGGING', 'DST_MOT#VAL=RUNNING', 'DST_PAC#VAL',
       'DST_SPD#VAL', 'EDA#VAL', 'HRT#VAL', 'RRI#VAL', 'SKT#VAL', 'STP#VAL',
       'ULV_INT#VAL=NONE', 'ULV_INT#VAL=LOW', 'ULV_INT#VAL=MEDIUM',
       'ULV_INT#VAL=HIGH', 'ULV_EXP#VAL', 'ACE_UNK#VAL', 'ACE_FOT#VAL',
       'ACE_WLK#VAL', 'ACE_VHC#VAL', 'ACE_BCC#VAL', 'ACE_RUN#VAL',
       'ACE_TLT#VAL', 'ACT#VAL=WALKING', 'ACT#VAL=STILL', 'ACT#VAL=IN_VEHICLE',
       'ACT#VAL=ON_BICYCLE', 'ACT#VAL=RUNNING', 'APP_DUR_UNKNOWN#VAL',
       'APP_DUR_SOCIAL#VAL', 'APP_DUR_SYSTEM#VAL', 'APP_DUR_HEALTH#VAL',
       'APP_DUR_ENTER#VAL', 'APP_DUR_INFO#VAL', 'APP_DUR_WORK#VAL',
       'APP_CAT#VAL=SOCIAL', 'APP_CAT#VAL=HEALTH', 'APP_CAT#VAL=ENTER',
       'APP_CAT#VAL=WORK', 'APP_CAT#VAL=INFO', 'BAT_LEV#VAL',
       'BAT_STA#VAL=CHARGING', 'BAT_STA#VAL=DISCHARGING', 'BAT_STA#V

# Evaluation

## Implementation

In [11]:
import numpy as np
from typing import Dict
from itertools import product
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, balanced_accuracy_score, \
    confusion_matrix, precision_recall_fscore_support, \
    roc_auc_score, matthews_corrcoef, average_precision_score, \
    log_loss, brier_score_loss
import scipy.stats.mstats as ms


def evaluate(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    y_prob: np.ndarray,
    classes: np.ndarray
) -> Dict[str, any]:
    R = {}
    n_classes = len(classes)
    is_multiclass = n_classes > 2
    is_same_y = len(np.unique(y_true)) == 1
    R['inst'] = len(y_true)
    
    for c in classes:
        R[f'inst_{c}'] = np.sum(y_true == c)
        
    if not is_multiclass:
        _, cnt = np.unique(y_true, return_counts=True)
        
        if len(cnt) > 1:
            R['class_ratio'] = cnt[0] / cnt[1]
        else:
            R['class_ratio'] = np.nan

    C = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=classes)
    for (i1, c1), (i2, c2) in product(enumerate(classes), enumerate(classes)):
        R[f'true_{c1}_pred_{c2}'] = C[i1, i2]

    # Threshold Measure
    R['acc'] = accuracy_score(y_true=y_true, y_pred=y_pred)
    R['bac'] = balanced_accuracy_score(y_true=y_true, y_pred=y_pred)
    R['gmean'] = ms.gmean(np.diag(C) / np.sum(C, axis=1))
    R['mcc'] = matthews_corrcoef(y_true=y_true, y_pred=y_pred)
    
    if is_multiclass:
        for avg in ('macro', 'micro'):
            pre, rec, f1, _ = precision_recall_fscore_support(
                y_true=y_true,
                y_pred=y_pred,
                labels=classes,
                average=avg, 
                zero_division=0
            )
            R[f'pre_{avg}'] = pre
            R[f'rec_{avg}'] = rec
            R[f'f1_{avg}'] = f1
    else:
        pre, rec, f1, _ = precision_recall_fscore_support(
            y_true=y_true, y_pred=y_pred, pos_label=c, average='macro', zero_division=0
        )
        R[f'pre_macro'] = pre
        R[f'rec_macro'] = rec
        R[f'f1_macro'] = f1
        
        for c in classes:
            pre, rec, f1, _ = precision_recall_fscore_support(
                y_true=y_true, y_pred=y_pred, pos_label=c, average='binary', zero_division=0
            )
            R[f'pre_{c}'] = pre
            R[f'rec_{c}'] = rec
            R[f'f1_{c}'] = f1

    # Ranking Measure
    if is_multiclass:
        for avg, mc in product(('macro', 'micro'), ('ovr', 'ovo')):
            R[f'roauc_{avg}_{mc}'] = roc_auc_score(
                y_true=y_true, y_score=y_prob,
                average=avg, multi_class=mc, labels=classes
            ) if not is_same_y else np.nan
    else:
        R[f'roauc'] = roc_auc_score(
            y_true=y_true, y_score=y_prob[:, 1], average=None
        ) if not is_same_y else np.nan
        for i, c in enumerate(classes):
            R[f'prauc_{c}'] = average_precision_score(
                y_true=y_true, y_score=y_prob[:, i], pos_label=c, average=None
            ) 
            R[f'prauc_ref_{c}'] = np.sum(y_true == c) / len(y_true)

    # Probability Measure
    R['log_loss'] = log_loss(y_true=y_true, y_pred=y_prob, labels=classes, normalize=True)

    if not is_multiclass:
        R[f'brier_loss'] = brier_score_loss(
            y_true=y_true, y_prob=y_prob[:, 1], pos_label=classes[1]
        )

    return R

## Execution

In [12]:
import os
import pandas as pd


RESULTS_EVAL = []
DIR_EVAL = os.path.join(PATH_INTERMEDIATE, 'eval')

#for l in ['valence', 'arousal', 'disturbance', 'stress']:
for l in [ 'stress']:
    dir_l = os.path.join(DIR_EVAL, l)
    if not os.path.exists(dir_l):
        continue
    
    for f in os.listdir(dir_l):
        model, pid = f[:f.index('.pkl')].split('#')
        res = load(os.path.join(dir_l, f))
        X, y = res.X_test, res.y_test
        y_pred = res.estimator.predict(X)
        y_prob = res.estimator.predict_proba(X)
        ev_test = evaluate(
            y_true=y,
            y_pred=y_pred,
            y_prob=y_prob,
            classes=[0, 1]
        )

        X, y = res.X_train, res.y_train
        y_pred = res.estimator.predict(X)
        y_prob = res.estimator.predict_proba(X)
        ev_train = evaluate(
            y_true=y,
            y_pred=y_pred,
            y_prob=y_prob,
            classes=[0, 1]
        )

        RESULTS_EVAL.append({
            'label': l,
            'alg': model,
            'split': pid,
            'n_feature': len(X.columns),
            **{
                f'test_{k}': v for k, v in ev_test.items()
            },
            **{
                f'train_{k}': v for k, v in ev_train.items()
            }
        })
    
RESULTS_EVAL = pd.DataFrame(RESULTS_EVAL)
RESULTS_EVAL.head()

,label,alg,split,n_feature,test_inst,test_inst_0,test_inst_1,test_class_ratio,test_true_0_pred_0,test_true_0_pred_1,...,train_pre_1,train_rec_1,train_f1_1,train_roauc,train_prauc_0,train_prauc_ref_0,train_prauc_1,train_prauc_ref_1,train_log_loss,train_brier_loss
0,stress,rf_ns,P42,229,70,26,44,0.590909,12,14,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.485681,1.000000,0.514319,0.139277,0.022851
1,stress,xgb_ns,P55,220,56,31,25,1.240000,23,8,...,0.951351,0.926316,0.938667,0.982113,0.979898,0.481077,0.984605,0.518923,0.209922,0.055866
2,stress,rf_ns,P45,213,38,11,27,0.407407,5,6,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.485471,1.000000,0.514529,0.135375,0.022176
3,stress,dummy,P77,6220,44,21,23,0.913043,0,21,...,0.517282,1.000000,0.681853,0.500000,0.482718,0.482718,0.517282,0.517282,0.692550,0.249701
4,stress,rf_ns,P28,219,83,2,81,0.024691,0,2,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.497634,1.000000,0.502366,0.139275,0.022833


In [13]:
import pandas as pd


SUMMARY_EVAL = []

for row in RESULTS_EVAL.groupby(
    ['label', 'alg']
).agg(summary).reset_index().itertuples():
    for k, v in row._asdict().items():
        if type(v) is dict:
            r = dict(
                label=row.label,
                alg=row.alg,
                metric=k,
                **v
            )
            SUMMARY_EVAL.append(r)

SUMMARY_EVAL = pd.DataFrame(SUMMARY_EVAL)    
SUMMARY_EVAL.head()

,label,alg,metric,n,cardinality,value_count,sum,mean,SD,med,range,conf.,nan_count
0,stress,dummy,split,47,47.0,"P77:1, P13:1, P60:1, P80:1, P61:1, P23:1, P55:...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,stress,dummy,n_feature,47,NaN,NaN,292340.0,6220.000000,0.000000,6220.0,"(6220, 6220)","(nan, nan)",0.0
2,stress,dummy,test_inst,47,NaN,NaN,2619.0,55.723404,13.076202,52.0,"(36, 83)","(51.88408763344431, 59.56272087719398)",0.0
3,stress,dummy,test_inst_0,47,NaN,NaN,1264.0,26.893617,14.547126,26.0,"(2, 56)","(22.622420860143833, 31.16481318240936)",0.0
4,stress,dummy,test_inst_1,47,NaN,NaN,1355.0,28.829787,15.207720,25.0,"(6, 81)","(24.364633393179524, 33.29494107490558)",0.0


In [14]:
SUB_SUMMARY_EVAL = SUMMARY_EVAL.loc[
    lambda x: x['metric'].isin(
        ['n_feature', 'train_class_ratio', 'train_inst_0', 'train_inst_1', 'test_inst_0', 'test_inst_1', 'test_acc', 'test_f1_0' ,'test_f1_1', 'test_f1_macro', 'train_f1_0' ,'train_f1_1', 'train_f1_macro',]
    )
].round(3).assign(
    mean_sd=lambda x: x['mean'].astype(str).str.cat(' (' + x['SD'].astype(str) + ')', sep='')
).pivot(
    index=['label', 'alg'], columns=['metric'], values=['mean_sd']
)
SUB_SUMMARY_EVAL

mean_sd                                               \
metric            n_feature       test_acc      test_f1_0      test_f1_1   
label  alg                                                                 
stress dummy   6220.0 (0.0)  0.516 (0.223)      0.0 (0.0)    0.652 (0.2)   
       rf_ns   221.0 (7.36)   0.786 (0.08)    0.7 (0.204)  0.718 (0.174)   
       xgb_ns  221.0 (7.36)  0.809 (0.071)  0.732 (0.204)  0.758 (0.136)   

                                                               \
metric         test_f1_macro      test_inst_0     test_inst_1   
label  alg                                                      
stress dummy     0.326 (0.1)  26.894 (14.547)  28.83 (15.208)   
       rf_ns   0.709 (0.087)  26.894 (14.547)  28.83 (15.208)   
       xgb_ns  0.745 (0.089)  26.894 (14.547)  28.83 (15.208)   

                                                                             \
metric        train_class_ratio    train_f1_0     train_f1_1 train_f1_macro   
label  alg                                                                    
stress dummy       0.933 (0.02)     0.0 (0.0)  0.682 (0.005)  0.341 (0.002)   
       rf_ns       0.933 (0.02)     1.0 (0.0)      1.0 (0.0)      1.0 (0.0)   
       xgb_ns      0.933 (0.02)  0.932 (0.01)   0.934 (0.01)   0.933 (0.01)   

                                                    
metric              train_inst_0      train_inst_1  
label  alg                                          
stress dummy   1237.106 (14.547)  1326.17 (15.208)  
       rf_ns   1237.106 (14.547)  1326.17 (15.208)  
       xgb_ns  1237.106 (14.547)  1326.17 (15.208)

# Feature Importances

## Implementation

In [ ]:
from typing import Union, Optional


def feature_importance(
    estimator
):
    if hasattr(estimator, 'model'):
        estimator = estimator.model
    
    if not hasattr(estimator, 'feature_names_in_') or not hasattr(estimator, 'feature_importances_'):
        return None
    
    names = estimator.feature_names_in_
    importances = estimator.feature_importances_
    
    return names, importances

## Execution

In [ ]:
import os
import pandas as pd
from collections import defaultdict


IMPORTANCE_EVAL = defaultdict(list)
DIR_EVAL = os.path.join(PATH_INTERMEDIATE, 'eval')


for f in os.listdir(DIR_EVAL):
    res = load(os.path.join(DIR_EVAL, f))
    
    f_norm = f[:f.index('.pkl')]
    cv = f_norm[:f.rindex('#')]
    
    feat_imp = feature_importance(res.estimator)
    if not feat_imp:
        continue
        
    names, importance = feat_imp
    d = pd.DataFrame(
        importance.reshape(1, -1),
        columns=names
    )
    IMPORTANCE_EVAL[cv].append(d)


IMPORTANCE_SUMMARY = []

for k, v in IMPORTANCE_EVAL.items():
    l, o, a = k.split('#')
    
    new_v = pd.concat(
        v, axis=0
    ).fillna(0.0).mean().reset_index().set_axis(
        ['feature', 'importance'], axis=1
    ).assign(
        label=l,
        oversampling=o,
        alg=a
    )
    IMPORTANCE_SUMMARY.append(new_v)
    
IMPORTANCE_SUMMARY = pd.concat(IMPORTANCE_SUMMARY, axis=0)
IMPORTANCE_SUMMARY.head()

### Plot

In [ ]:
%%R -i IMPORTANCE_SUMMARY -w 26 -h 22 -u cm

plots <- list()

#for (l in c('valence', 'arousal', 'stress', 'disturbance')) {
for (l in c( 'stress')) {
    data <- IMPORTANCE_SUMMARY %>% filter(
        (label == l) & (oversampling == 'os')
    )

    p_label <- ggplot() + geom_text(
        aes(x=.5, y=.5),
        label=str_to_title(l), 
        family='ssp', 
        fontface='bold',
        size=4
    ) + theme_void()

    data_rf <- data %>% filter(
        alg == 'rf'
    )

    p_rf <- ggplot(
        data %>% filter(alg == 'rf') %>% top_n(n=10, wt=importance),
        aes(x=reorder(feature, -importance), y=importance),
    ) + geom_col(
    ) + THEME_DEFAULT + theme(
        axis.text.x=element_text(angle=90, size=10, hjust=1),
        axis.title.x=element_blank(),
        axis.title.y=element_blank()
    ) + labs(
        subtitle='Random Forest'
    ) + ylim(
        c(0, 0.08)
    )
    p_xgb <- ggplot(
        data %>% filter(alg == 'xgb') %>% top_n(n=10, wt=importance),
        aes(x=reorder(feature, -importance), y=importance),
    ) + geom_col(
    ) + THEME_DEFAULT + theme(
        axis.text.x=element_text(angle=90, size=10, hjust=1),
        axis.title.x=element_blank(),
        axis.title.y=element_blank()
    ) + labs(
        subtitle='XGBoost'
    ) + ylim(
        c(0, 0.08)
    )
    plots[[paste(l, 'label', sep='_')]] <- p_label
    plots[[paste(l, 'rf', sep='_')]] <- p_rf
    plots[[paste(l, 'xgb', sep='_')]] <- p_xgb
}

#p <- plots$arousal_label + plots$valence_label
#p <- p / (plots$arousal_rf | plots$arousal_xgb | plots$valence_rf | plots$valence_xgb)
#p <- p / (plots$stress_label + plots$disturbance_label)
#p <- p / (plots$stress_rf | plots$stress_xgb | plots$disturbance_rf | plots$disturbance_xgb)
p <- plots$stress_label 
p <- p / (plots$stress_rf | plots$stress_xgb)


p <- p + plot_layout(
    heights=c(1.1, 10, 1.1, 10)
)

ggsave(paste('./fig/imp.pdf'), plot=p, width=26, height=20, unit='cm', device=cairo_pdf)
print(p)
